In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import use_hyperas
from tuner import net
from tuner.dataset import ClassificationDataset, AugmentDataset

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from easydict import EasyDict
params = EasyDict({
    'src_dir': '../micin-dataset/brain',
    'dst_dir': './brain'
})

In [3]:
df = load_data.df_fromdir_brain(params.src_dir)

In [4]:
load_data.classed_dir_fromdf(df, "tmp/brain")

In [5]:
brain = ClassificationDataset('tmp/brain')

In [6]:
brain_dataset = brain

In [7]:
min(brain_dataset.counts_train_data().values())

51

In [8]:
aug_brain = AugmentDataset(brain_dataset)

Initialised with 246 image(s) found.
Output directory set to standard_datasets/5a5f131db037cb8223459336/train/output.

In [9]:
aug_brain.augmented_dir

'standard_datasets/5a5f131db037cb8223459336/auged'

In [10]:
aug_brain.dataset.train_dir

'standard_datasets/5a5f131db037cb8223459336/train'

In [11]:
aug_brain.search_opt_augment(model=net.neoaug)

def data():
    train_dir = '{train_dir}' 
    test_dir = '{validation_dir}'
    resize = 96 
    rescale = 1 
    df = load_data.df_fromdir(train_dir)
    x_train, y_train = load_data.load_fromdf(df, resize=resize, rescale=rescale)
    df = load_data.df_fromdir(test_dir)
    x_test, y_test = load_data.load_fromdf(df, resize=resize, rescale=rescale)

    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)

    return x_train, y_train, x_test, y_test

         name label        handle  \
0   MS163.jpg    MS  MS/MS163.jpg   
1   MS104.jpg    MS  MS/MS104.jpg   
2   MS139.jpg    MS  MS/MS139.jpg   
3   MS161.jpg    MS  MS/MS161.jpg   
4   MS159.jpg    MS  MS/MS159.jpg   
5   MS140.jpg    MS  MS/MS140.jpg   
6   MS105.jpg    MS  MS/MS105.jpg   
7   MS162.jpg    MS  MS/MS162.jpg   
8    MS96.jpg    MS   MS/MS96.jpg   
9   MS125.jpg    MS  MS/MS125.jpg   
10   MS83.jpg    MS   MS/MS83.jpg   
11  MS070.jpg    MS  MS/MS070.jpg   
12  MS116.jpg    MS  MS/MS116.jpg   
13  MS117

IsADirectoryError: [Errno 21] Is a directory: 'standard_datasets/5a5f131db037cb8223459336/train/output/MS'

In [1]:
aug_brain.augment_dataset()

NameError: name 'aug_brain' is not defined

In [12]:
train_dir =  'standard_datasets/5a5f131db037cb8223459336/train' 

In [18]:
load_data.df_fromdir(train_dir).path[2]

'standard_datasets/5a5f131db037cb8223459336/train/MS/MS139.jpg'

In [ ]:
best_condition, best_model = use_hyperas.exec_hyperas(
    aug_brain.augmented_dir,
    aug_brain.dataset.validation_dir,
    net.simplenet,
    batch_size = 32,
    epochs=100,
    optimizer='adam'
)
fname = 'simplenet.hdf5'
best_model.save(fname)

In [25]:
import pandas as pd

In [43]:
def fff(dataset_dir, columns=['name', 'label']):
    fname_label = []

    labels = load_data.get_labels_fromdir(dataset_dir)

    for label in labels:
        for fname in os.listdir(os.path.join(dataset_dir, label)):
            d = (fname, label)
            fname_label.append(d)
    df = pd.DataFrame(fname_label, columns=columns)
    df['handle'] = utils.path_join(df['label'], df['name'])
    df['path'] = utils.path_join(dataset_dir, df['label'], df['name'])
    return df

In [44]:
df = fff(train_dir)

In [50]:
fff

<function __main__.fff>

In [45]:
import inspect

In [49]:
code = inspect.getsource(load_data.df_fromdir)
print(code)

def get_labels_fromdir(dataset_dir):
    # hyperas don't allowed itertor
    #can = next(os.walk(dataset_dir))[1]

    can = os.listdir(dataset_dir)
    can = [c for c in can if os.path.isdir(os.path.join(dataset_dir, c))]
    return [c for c in can if c != 'output']



In [41]:
df - load_data.df_fromdir(train_dir)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [40]:
can

['MS', 'PD', 'N', 'PS', 'output']

In [38]:
os.path.isdir('/')

True

In [31]:
load_data.get_labels_fromdir(train_dir)


['MS', 'PD', 'N', 'PS', 'output']

In [29]:
df - fff(train_dir)

TypeError: unsupported operand type(s) for -: 'str' and 'str'